# **1.모듈 불러오기**

In [1]:
# 데이터 프레임
import pandas as pd

# datetime : 날짜 및 시간 다루기 그 중 date를 다룸
# relativedelta : 날짜 간의 상대적인 차이를 계산
from datetime import date 
from dateutil.relativedelta import relativedelta

# requests : HTTP 요청을 보내기 위한  
# BeautifulSoup : 라이브러리 및 웹 스크래핑을 위한
import requests
from bs4 import BeautifulSoup

# 시간 지연
import time 

# **2. 함수 만들기**

In [2]:
def convert_ym(start_ymd: str, months: int) -> list:
    """iso 표준 형식(yyyy-mm)의 날짜 문자열 변환.

    Args:
        start_ymd: yyyy-mm-dd 형식의 시작 년월일을 나타내는 문자열.
        months: 날짜 생성을 위한 반복 개월 수.

    Returns:
        년월(yyyymm) 문자열을 원소하는 리스트
        예:

        ['202001', '202002', '202003', '202004', '202005']
    """

    start_date = date.fromisoformat(start_ymd)           # 시작날짜 (iso 날짜 표준 : yyyy-mm-dd)
    yyyymm = []   
                                           
    for i in range(months) :                              
        d = start_date + relativedelta(months=i)         # 시작날짜 + i개월을 더한 날짜 생성
        yyyymm.append("{}{:02}".format(d.year, d.month)) # yyyymm 리스트에 항목(년도 4자리 월 2자리로 구성된 문자열) 추가

    return yyyymm      

# **3. API 요청변수**

In [4]:
# api_mykey = 공공데이터포털 개인 API KEY 입력해야 오류나지 않음
landcode = "11110"

# 2022년 1월 1일 부터 16개월 -> yyyymm 형식으로 데이터를 리스트에 저장함
yms = convert_ym("2022-01-01", 16)


r_type = "json"
lte = "202305"
gte = "202201"

# **4. 데이터 스크래핑**

## 1) 아파트 매매 실거래가 수집

In [7]:
# 데이터를 저장할 빈 리스트
pur_data = []


# yms만큼 반복하여 작업
for ym in yms :
    
    # API 요청을 위한 URL 생성
    pur_url = "http://openapi.molit.go.kr:8081/OpenAPI_ToolInstallPackage/service/rest/RTMSOBJSvc/getRTMSDataSvcAptTrade?serviceKey={}&LAWD_CD={}&DEAL_YMD={}"
    pur_url = pur_url.format(api_mykey, landcode, ym)

    # 생성한 URL GET요청
    pur_res = requests.get(pur_url)
    # 응답 데이터를 XML 형식으로 파싱
    pur_bs = BeautifulSoup(pur_res.text, 'xml')

    # XML에서 'item' 요소들을 찾음
    items = pur_bs.find_all('item')

    for item in items : 

        pur_result = [] 
        
         # 날짜 정보를 '년', '월', '일' 요소에서 추출하여 YYYY-MM-DD 형식으로 저장
        date = "{}-{:02}-{:02}".format(item.find('년').string.strip(), int(item.find('월').string.strip()), int(item.find('일').string.strip()))
        
        # pur_result 리스트에 저장
        pur_result.append(date)
        # pur_result.append(item.find('년').string.strip())
        # pur_result.append(item.find('월').string.strip())
        # pur_result.append(item.find('일').string.strip())
        pur_result.append(item.find('법정동').string.strip())
        pur_result.append(item.find('지번').string.strip())
        pur_result.append(item.find('아파트').string.strip())
        pur_result.append(item.find('층').string.strip())
        pur_result.append(item.find('전용면적').string.strip())
        pur_result.append(item.find('거래금액').string.strip())
        pur_result.append(item.find('건축년도').string.strip())

        # pur_result 리스트에 담긴 값을 pur_data 리스트로
        pur_data.append(pur_result)

# 데이터 처리를 위해 2초 대기
time.sleep(2)


## 1-2) 매매가 데이터 프레임 만들기

In [8]:
# pur_data를 컬럼명을 지정하여 pur_df 데이터프레임으로 저장
col = ['날짜', '법정동', '지번', '아파트', '층', '전용면적', '거래금액', '건축년도']
pur_df = pd.DataFrame(pur_data, columns=col)

# 거래금액 값에 ,를 없앤 후 int로 변환
pur_df['거래금액'] = pur_df['거래금액'].str.replace(',', '').astype(int)

# 날짜를 index로 지정
pur_df.set_index("날짜")

,법정동,지번,아파트,층,전용면적,거래금액,건축년도
날짜,,,,,,,
2022-01-14,내수동,72,경희궁의아침3단지,3,150.48,200000,2004
2022-01-11,연지동,163-3,아르젠종로,10,18.02,20000,2015
2022-01-08,효제동,65-2,포레스트힐시티,6,16.672,19000,2017
2022-01-03,연건동,195-10,이화에수풀,7,16.98,19500,2014
2022-01-08,연건동,195-10,이화에수풀,11,16.98,19000,2014
...,...,...,...,...,...,...,...
2023-04-14,홍파동,199,경희궁자이(2단지),10,84.8359,180200,2017
2023-04-23,홍파동,199,경희궁자이(2단지),14,59.8547,155000,2017
2023-04-27,홍파동,199,경희궁자이(2단지),17,77.9737,162500,2017


## 1-3) 매매가 CSV로 저장하기

In [9]:
pur_df.to_csv('아파트매매가_data(2022~2023).csv', index=False)

## 2) 아파트 전세가 수집  

In [10]:
jeonse_data = [] 

for ym in yms:
   
    jeonse_url = "http://openapi.molit.go.kr:8081/OpenAPI_ToolInstallPackage/service/rest/RTMSOBJSvc/getRTMSDataSvcAptRent?serviceKey={}&LAWD_CD={}&DEAL_YMD={}"
    jeonse_url = jeonse_url.format(api_mykey, landcode, ym)
    jeonse_res = requests.get(jeonse_url)
    jeonse_bs = BeautifulSoup(jeonse_res.text, 'xml')

    items = jeonse_bs.find_all('item')

    for item_j in items : 

        jeonse_result = [] 
        
        if item_j.find('월세금액').string.strip() == '0':
            jeonse_result.append('{}-{:02}-{:02}'.format(item_j.find('년').string.strip(), int(item_j.find('월').string.strip()), int(item_j.find('일').string.strip())))
            # jeonse_result.append(item_j.find('년').string.strip())
            # jeonse_result.append(item_j.find('월').string.strip())
            # jeonse_result.append(item_j.find('일').string.strip())
            jeonse_result.append(item_j.find('법정동').string.strip())
            jeonse_result.append(item_j.find('지번').string.strip())
            jeonse_result.append(item_j.find('아파트').string.strip())
            jeonse_result.append(item_j.find('층').string.strip())
            jeonse_result.append(item_j.find('전용면적').string.strip())
            jeonse_result.append(item_j.find('보증금액').string.strip())
            #jeonse_result.append(item_j.find('월세금액').string.strip())
            jeonse_result.append(item_j.find('건축년도').string.strip())

            jeonse_data.append(jeonse_result)

time.sleep(2)

## 2-2) 전세가 데이터 프레임 만들기

In [11]:
col_j= ['날짜', '법정동', '지번', '아파트', '층', '전용면적', '보증금액', '건축년도']
jeonse_df = pd.DataFrame(jeonse_data, columns=col_j)
jeonse_df['보증금액'] = jeonse_df['보증금액'].str.replace(',', '').astype(int)
jeonse_df.set_index('날짜')

,법정동,지번,아파트,층,전용면적,보증금액,건축년도
날짜,,,,,,,
2022-01-04,사직동,9,광화문스페이스본(101동~105동),9,147.31,97650,2008
2022-01-07,사직동,9,광화문스페이스본(101동~105동),1,151.81,145000,2008
2022-01-28,사직동,9,광화문스페이스본(101동~105동),11,97.61,80000,2008
2022-01-22,신문로1가,238,신문로맨션,8,98.31,50000,1981
2022-01-13,익선동,55,현대뜨레비앙,7,36.08,21000,2003
...,...,...,...,...,...,...,...
2023-04-15,무악동,88,인왕산2차아이파크,3,84.2395,65000,2015
2023-04-15,무악동,89,경희궁롯데캐슬,4,84.8792,82000,2019
2023-04-16,무악동,60,인왕산아이파크,17,84.858,61000,2008


## 2-3) 전세가 CSV로 저장하기

In [12]:
jeonse_df.to_csv('아파트전세가_data(2022~2023).csv', index=False)

## 3) 지역별 생활업종 현황 수집

In [13]:
life_df = pd.read_csv("download.csv", encoding="euc-kr")
life_df.to_csv("life_infra", index=None)
life_df

,업종,시도,시군구,당월,전월,전년동월
0,가구점,서울특별시,종로구,22.0,22.0,19.0
1,가구점,서울특별시,중구,65.0,66.0,69.0
2,가구점,서울특별시,용산구,39.0,39.0,35.0
3,가구점,서울특별시,성동구,28.0,28.0,27.0
4,가구점,서울특별시,광진구,38.0,38.0,41.0
...,...,...,...,...,...,...
23851,주차장운영업,경상남도,산청군,3.0,3.0,3.0
23852,주차장운영업,경상남도,거창군,7.0,7.0,6.0
23853,주차장운영업,경상남도,합천군,NaN,NaN,NaN
23854,주차장운영업,제주특별자치도,제주시,70.0,70.0,57.0


## 3-2) 지역별 생활업종 CSV로 저장하기

In [14]:
life_df.to_csv("생활업종_data.csv", index=None)

## 4) 부동산 매매 거래건수 수집

In [15]:
cnt_data = [] 

   
cnt_url = "https://api.odcloud.kr/api/RealEstateTradingSvc/v1/getRealEstateTradingCount?page=1&perPage=10000&returnType={}&cond%5BRESEARCH_DATE%3A%3ALTE%5D={}&cond%5BRESEARCH_DATE%3A%3AGTE%5D={}&cond%5BREGION_CD%3A%3AEQ%5D={}&cond%5BDEAL_OBJ%3A%3AEQ%5D=07&serviceKey={}"
cnt_url = cnt_url.format(r_type, lte, gte, landcode, api_mykey)

json_cnt_data = requests.get(cnt_url).json()
# print(json_cnt_data)
# print(json_cnt_data.keys())

for element in json_cnt_data["data"]:

   cnt_result = []


   year_cnt = element["RESEARCH_DATE"][:4]
   month_cnt = element["RESEARCH_DATE"][4:]

   cnt_result.append("{}-{}".format(year_cnt, month_cnt))
   cnt_result.append(element["ALL_CNT"])
   cnt_data.append(cnt_result)
   # print("RESEARCH_DATE: {}, ALL_CNT: {}".format(element["RESEARCH_DATE"], element["ALL_CNT"]))

### 4-2. 부동산 매매 거래건수 데이터 프레임 만들기

In [16]:
col_c = ['RESEARCH_DATE', 'ALL_CNT']
cnt_df = pd.DataFrame(cnt_data, columns=col_c)
cnt_df

,RESEARCH_DATE,ALL_CNT
0,2022-02,15
1,2022-03,22
2,2022-04,25
3,2022-01,25
4,2022-06,18
5,2022-05,29
6,2022-09,14
7,2022-07,11
8,2022-10,8
9,2022-08,16


### 4-3. 부동산 매매 거래건수 CSV로 저장하기

In [17]:
cnt_df.to_csv('매매건수_data(2022~2023).csv', index=False)